<a href="https://colab.research.google.com/github/Pragati-cloud/PythonBasic/blob/main/Movie_sentiment_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# runtime: GPU recommended (Runtime -> Change runtime type -> GPU)
!nvidia-smi

# Install libraries
!pip install -q tensorflow==2.20.0 pandas scikit-learn nltk tqdm requests
# optional: tensorflow-addons might be used as fallback for AdamW
!pip install -q tensorflow-addons


/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 126.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons


In [4]:
from google.colab import drive
drive.mount('/content/drive')
# Optional: set a path in your Drive where models/tokenizer will be saved
DRIVE_OUT = "/content/drive/MyDrive/sentiment_imdb"


MessageError: Error: credential propagation was unsuccessful

In [5]:
from google.colab import files
uploaded = files.upload()  # use the file browser to upload your CSV (select the Kaggle IMDB CSV)
# after upload, check filename
for fn in uploaded.keys():
    print("Uploaded:", fn)
DATA_CSV_PATH = list(uploaded.keys())[0]


Saving IMDB Dataset.csv.zip to IMDB Dataset.csv.zip
Uploaded: IMDB Dataset.csv.zip


In [6]:
import os, re, io, zipfile, requests
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import pickle
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def custom_standardization(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)           # remove html
    text = re.sub(r'http\S+|www\.\S+', ' ', text) # remove urls
    text = re.sub(r'[^a-z\s]', ' ', text)        # non-letter chars -> space
    text = re.sub(r'\s+', ' ', text).strip()     # collapse spaces
    tokens = [ps.stem(tok) for tok in text.split() if tok not in STOPWORDS]
    return ' '.join(tokens)

# Hyperparams (you can tweak)
MODEL_DIR = "/content/model"
os.makedirs(MODEL_DIR, exist_ok=True)

VOCAB_SIZE = 20000
MAX_LEN = 300
EMBED_DIM = 100   # your chosen 100d
BATCH_SIZE = 128
EPOCHS = 10
TEST_SIZE = 0.2
RANDOM_STATE = 42

GLOVE_URL = "https://nlp.stanford.edu/data/glove.6B.zip"
GLOVE_DIR = "/content/glove"
os.makedirs(GLOVE_DIR, exist_ok=True)


In [8]:
print("Loading CSV:", DATA_CSV_PATH)
df = pd.read_csv(DATA_CSV_PATH)

# Accept common variants for column names
if 'review' not in df.columns or 'sentiment' not in df.columns:
    if 'text' in df.columns and 'label' in df.columns:
        df = df.rename(columns={'text': 'review', 'label': 'sentiment'})
    elif 'review' in df.columns and 'label' in df.columns:
        df = df.rename(columns={'label': 'sentiment'})
    else:
        print("Columns found:", df.columns)
        raise ValueError("CSV must contain 'review' and 'sentiment' columns (or 'text'/'label').")

# Map textual sentiments ('positive'/'negative') to 1/0 if needed
if df['sentiment'].dtype == object:
    df['sentiment'] = df['sentiment'].str.strip().str.lower()
    mapping = {'positive': 1, 'negative': 0, 'pos': 1, 'neg': 0}
    # if the column contains 'positive'/'negative' strings, map them
    if df['sentiment'].isin(['positive','negative','pos','neg']).all():
        df['sentiment'] = df['sentiment'].map(mapping)
    else:
        # try to convert to numeric (handles '1'/'0' strings)
        df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')

# final sanity checks
if df['sentiment'].isnull().any():
    raise ValueError("Some sentiment labels couldn't be converted to ints. Unique values: "
                     + str(df['sentiment'].unique()))

df['sentiment'] = df['sentiment'].astype(int)
print("Loaded dataset size:", len(df))
print("Label distribution:\n", df['sentiment'].value_counts())


Loading CSV: IMDB Dataset.csv.zip
Loaded dataset size: 50000
Label distribution:
 sentiment
1    25000
0    25000
Name: count, dtype: int64


In [9]:
print("Cleaning reviews (may take a few minutes)...")
df['clean_review'] = df['review'].astype(str).apply(custom_standardization)

X = df['clean_review'].values
y = df['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

print("Train size:", len(X_train), "Test size:", len(X_test))


Cleaning reviews (may take a few minutes)...
Train size: 40000 Test size: 10000


In [10]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# save tokenizer
with open(os.path.join(MODEL_DIR, "tokenizer.pkl"), "wb") as f:
    pickle.dump(tokenizer, f)
print("Tokenizer saved to", os.path.join(MODEL_DIR, "tokenizer.pkl"))

train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(test_seq, maxlen=MAX_LEN, padding='post', truncating='post')
print("Padded shapes:", X_train_pad.shape, X_test_pad.shape)


Tokenizer saved to /content/model/tokenizer.pkl
Padded shapes: (40000, 300) (10000, 300)


In [11]:
import zipfile, requests, io
glove_target = os.path.join(GLOVE_DIR, f"glove.6B.{EMBED_DIM}d.txt")

if not os.path.exists(glove_target):
    print("Downloading GloVe... (~862MB zip, will extract only the chosen file)")
    r = requests.get(GLOVE_URL, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('content-length', 0))
    buf = io.BytesIO()
    for chunk in tqdm(r.iter_content(chunk_size=1024*1024), total=total//(1024*1024), desc="Downloading"):
        if chunk:
            buf.write(chunk)
    buf.seek(0)
    with zipfile.ZipFile(buf) as z:
        member = f"glove.6B.{EMBED_DIM}d.txt"
        print("Extracting", member)
        z.extract(member, path=GLOVE_DIR)

print("GloVe file at:", glove_target)


Downloading: 823it [02:39,  5.18it/s]                         


Extracting glove.6B.100d.txt
GloVe file at: /content/glove/glove.6B.100d.txt


In [12]:
glove_path = os.path.join(GLOVE_DIR, f"glove.6B.{EMBED_DIM}d.txt")
emb_index = {}
print("Loading GloVe vectors (this may take a minute)...")
with open(glove_path, 'r', encoding='utf8') as f:
    for line in tqdm(f, desc="Reading GloVe"):
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        emb_index[word] = vec
print("GloVe vectors:", len(emb_index))

word_index = tokenizer.word_index
num_tokens = min(VOCAB_SIZE, len(word_index) + 1)
embedding_matrix = np.zeros((num_tokens, EMBED_DIM), dtype=np.float32)

for word, i in word_index.items():
    if i >= num_tokens:
        continue
    vec = emb_index.get(word)
    if vec is not None:
        embedding_matrix[i] = vec
    else:
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(EMBED_DIM,))

print("Embedding matrix shape:", embedding_matrix.shape)


Loading GloVe vectors (this may take a minute)...


Reading GloVe: 400000it [00:12, 33062.54it/s]


GloVe vectors: 400000
Embedding matrix shape: (20000, 100)


In [13]:
from tensorflow.keras import regularizers

model = Sequential([
    Embedding(num_tokens, EMBED_DIM,
              embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
              input_length=MAX_LEN, trainable=False),
    SpatialDropout1D(0.3),
    Bidirectional(LSTM(128, return_sequences=False)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

# AdamW fallback logic (you requested AdamW)
try:
    from tensorflow.keras.optimizers import AdamW
    opt = AdamW(learning_rate=1e-3)
    print("Using tf.keras.optimizers.AdamW")
except Exception:
    try:
        import tensorflow_addons as tfa
        opt = tfa.optimizers.AdamW(weight_decay=1e-5, learning_rate=1e-3)
        print("Using tensorflow_addons.AdamW")
    except Exception:
        opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
        print("AdamW not found; using Adam as fallback")

model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Using tf.keras.optimizers.AdamW


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
checkpoint_path = os.path.join(MODEL_DIR, "sentiment_model_best.keras")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)
]

history = model.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6031 - loss: 0.6568
Epoch 1: val_loss improved from inf to 0.57331, saving model to /content/model/sentiment_model_best.keras
282/282 ━━━━━━━━━━━━━━━━━━━━ 433s 2s/step - accuracy: 0.6032 - loss: 0.6567 - val_accuracy: 0.7180 - val_loss: 0.5733 - learning_rate: 0.0010
Epoch 2/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6921 - loss: 0.5936
Epoch 2: val_loss improved from 0.57331 to 0.49566, saving model to /content/model/sentiment_model_best.keras
282/282 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step - accuracy: 0.6921 - loss: 0.5935 - val_accuracy: 0.7703 - val_loss: 0.4957 - learning_rate: 0.0010
Epoch 3/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7334 - loss: 0.5423
Epoch 3: val_loss improved from 0.49566 to 0.43304, saving model to /content/model/sentiment_model_best.keras
282/282 ━━━━━━━━━━━━━━━━━━━━ 428s 2s/step - accuracy: 0.7334 - loss: 0.5423 - val_accuracy: 0.8030 - val_loss: 0.4330 - learning_rate: 

In [15]:
final_model_path = os.path.join(MODEL_DIR, "sentiment_model.keras")
model.save(final_model_path)
print("Saved model to", final_model_path)

# also copy to Drive if mounted
try:
    import shutil
    dst = os.path.join(DRIVE_OUT, "model")
    os.makedirs(dst, exist_ok=True)
    shutil.copy(final_model_path, dst)
    shutil.copy(os.path.join(MODEL_DIR, "tokenizer.pkl"), dst)
    print("Copied model+tokenizer to Drive at", dst)
except Exception as e:
    print("Copy to Drive skipped or failed:", e)

# Evaluate
probs = model.predict(X_test_pad, batch_size=1024)
preds = np.argmax(probs, axis=1)
acc = accuracy_score(y_test, preds)
print("Test Accuracy: {:.4f}".format(acc))
print("\nClassification report:\n")
print(classification_report(y_test, preds, digits=4))


Saved model to /content/model/sentiment_model.keras
Copy to Drive skipped or failed: name 'DRIVE_OUT' is not defined
10/10 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step
Test Accuracy: 0.8597

Classification report:

              precision    recall  f1-score   support

           0     0.8849    0.8270    0.8550      5000
           1     0.8376    0.8924    0.8641      5000

    accuracy                         0.8597     10000
   macro avg     0.8612    0.8597    0.8595     10000
weighted avg     0.8612    0.8597    0.8595     10000



In [16]:
# load tokenizer and model to confirm inference works
with open(os.path.join(MODEL_DIR, "tokenizer.pkl"), "rb") as f:
    tok = pickle.load(f)

m = tf.keras.models.load_model(os.path.join(MODEL_DIR, "sentiment_model.keras"))

def predict_text(text):
    clean = custom_standardization(text)
    seq = tok.texts_to_sequences([clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding='post')
    prob = m.predict(pad)[0]
    return {"negative_prob": float(prob[0]), "positive_prob": float(prob[1]),
            "sentiment": "Positive" if prob[1] > 0.5 else "Negative"}

# test
print(predict_text("This was the best movie I have ever seen! I loved it."))
print(predict_text("I hated this movie. It was boring and a waste of time."))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
{'negative_prob': 0.02728801779448986, 'positive_prob': 0.972711980342865, 'sentiment': 'Positive'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
{'negative_prob': 0.9676733016967773, 'positive_prob': 0.032326724380254745, 'sentiment': 'Negative'}
